In [87]:
import pandas as pd
import pyarrow.parquet as pq
import gzip

In [88]:
steam1 = pd.read_csv('./Data\Data-Limpia\steam_games_limpio.csv')

items = pd.read_csv('./Data/Data-Limpia/user_items_limpio.csv')

steam1 = steam1[['genres','id','año']]
items = items[['user_id','item_id','playtime_forever']]

In [89]:
steam1 = steam1.groupby('id').first().reset_index()

steam1 = steam1[steam1['año']!=0]

resultado = steam1.merge(items, left_on='id', right_on='item_id', how='inner')


In [90]:
resultado

,id,genres,año,user_id,item_id,playtime_forever
0,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,doctr,10.0,1.550000
1,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,corrupted_soul,10.0,1.800000
2,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,WeiEDKrSat,10.0,5.466667
3,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,death-hunter,10.0,104.583333
4,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,Fr0stedLine,10.0,9.666667
...,...,...,...,...,...,...
2022223,527340.0,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,2spookis,527340.0,1.266667
2022224,527340.0,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,76561198068423082,527340.0,1.966667
2022225,527340.0,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,jandango,527340.0,1.100000
2022226,527510.0,"['Casual', 'Simulation', 'Indie', 'Strategy']",2016,mayshowganmore,527510.0,2.566667


In [91]:
resultado['playtime_forever'] = resultado['playtime_forever'].astype(int)

In [92]:
resultado=resultado.drop(['id','item_id'],axis=1)

In [93]:
resultado

,genres,año,user_id,playtime_forever
0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,doctr,1
1,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,corrupted_soul,1
2,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,WeiEDKrSat,5
3,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,death-hunter,104
4,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,Fr0stedLine,9
...,...,...,...,...
2022223,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,2spookis,1
2022224,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,76561198068423082,1
2022225,"['Action', 'Indie', 'Multiplayer', 'Funny', 'A...",2016,jandango,1
2022226,"['Casual', 'Simulation', 'Indie', 'Strategy']",2016,mayshowganmore,2


In [94]:
with gzip.open('./Data/Data-Funciones/F_user_genre.csv.gz', 'wb') as f:
    resultado.to_csv(f, index=False, encoding='utf-8')

In [42]:
df = pd.read_csv('./Data/Data-Funciones/F_user_genre.csv.gz', compression='gzip')


In [95]:
def UserForGenre2(genero):
    # Filtrar el DataFrame por el género dado
    
    df = pd.read_csv('./Data/Data-Funciones/F_user_genre.csv.gz', compression='gzip')

    df_genero = df[df['genres'].str.contains(genero)] 
    df_horas = df_genero.groupby('user_id')['playtime_forever'].sum().reset_index()


    # Encontrar el usuario con más horas jugadas para ese género
    usuario_max_playtime = df_horas.loc[df_horas['playtime_forever'].idxmax()]['user_id']


    # Filtrar el DataFrame por el usuario con más horas jugadas en ese género
    df_usuario_max = df_genero[df_genero['user_id'] == usuario_max_playtime]
    df_horas_anio = df_usuario_max.groupby('año')['playtime_forever'].sum().reset_index()

    horas_anio = [{'Año': row['año'], 'Horas': row['playtime_forever']} for _, row in df_horas_anio.iterrows()]
    salida = {'Usuario con más horas jugadas para ' + 'Action': usuario_max_playtime, 'Horas jugadas': horas_anio}
    return salida

In [96]:
UserForGenre2('Action')

{'Usuario con más horas jugadas para Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 3},
  {'Año': 1997, 'Horas': 4},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2001, 'Horas': 3},
  {'Año': 2002, 'Horas': 3},
  {'Año': 2003, 'Horas': 128},
  {'Año': 2004, 'Horas': 2122},
  {'Año': 2005, 'Horas': 355},
  {'Año': 2006, 'Horas': 1502},
  {'Año': 2007, 'Horas': 1878},
  {'Año': 2008, 'Horas': 141},
  {'Año': 2009, 'Horas': 1810},
  {'Año': 2010, 'Horas': 1374},
  {'Año': 2011, 'Horas': 2582},
  {'Año': 2012, 'Horas': 6389},
  {'Año': 2013, 'Horas': 2024},
  {'Año': 2014, 'Horas': 2199},
  {'Año': 2015, 'Horas': 6477},
  {'Año': 2016, 'Horas': 1988},
  {'Año': 2017, 'Horas': 720}]}